In [7]:
import pandas as pd
import numpy as np

# Load the cleaned dataset
df = pd.read_csv("ott_complete_cleaned_dataset.csv")

# Load original dataset to restore missing columns (if needed)
# original_df = pd.read_csv("raw_data.csv")

# Restore 'customer_id' and 'year' if they were dropped
if 'customer_id' not in df.columns or 'year' not in df.columns:
    df = df.merge(original_df[['customer_id', 'year']], on='customer_id', how='left')

# Convert text columns to lowercase
df['multi_screen'] = df['multi_screen'].astype(str).str.lower()
df['mail_subscribed'] = df['mail_subscribed'].astype(str).str.lower()

# Multi-Screen Score (2 = "Yes", 1 = "No")
df['multi_screen_score'] = df['multi_screen'].apply(lambda x: 2 if x == 'yes' else (1 if x == 'no' else np.nan))
df['multi_screen_score'] = df['multi_screen_score'].fillna(1)

# Email Marketing Score (2 = "Yes", 1 = "No")
df['email_marketing_score'] = df['mail_subscribed'].apply(lambda x: 2 if x == 'yes' else (1 if x == 'no' else np.nan))
df['email_marketing_score'] = df['email_marketing_score'].fillna(1)

# Subscription Tenure Score (0 to 1 scale)
df['subscription_tenure'] = df['no_of_days_subscribed'] / df['no_of_days_subscribed'].max()

# Engagement Score (Higher engagement = Higher score)
df['engagement_score'] = df['weekly_mins_watched'] / df['weekly_mins_watched'].max()

# Consistency Score
df['consistency_score'] = 1 - (df['maximum_daily_mins'] - df['minimum_daily_mins']) / df['maximum_daily_mins']
df['consistency_score'] = df['consistency_score'].fillna(0.5)

# Night Owl Score
df['night_owl_score'] = df['weekly_max_night_mins'] / df['weekly_mins_watched']
df['night_owl_score'] = df['night_owl_score'].fillna(0)

# Churn Risk Score
df['churn_risk_score'] = (df['customer_support_calls'] / df['customer_support_calls'].max()) + (1 - df['engagement_score'])
df['churn_risk_score'] = df['churn_risk_score'] / df['churn_risk_score'].max()

# Final Cleaning
df = df.fillna(0)

# Save processed data
df.to_csv("processed_data.csv", index=False)

# Summary
print(df.head())
print("Advanced Feature Engineering Completed Successfully!")


   year  customer_id  gender  age  no_of_days_subscribed multi_screen  \
0  2020       100198  Female   36                     62           no   
1  2020       100643  Female   39                    149           no   
2  2020       100756  Female   65                    126           no   
3  2020       101595  Female   24                    131           no   
4  2020       101653  Female   40                    191           no   

  mail_subscribed  weekly_mins_watched  minimum_daily_mins  \
0              no               148.35                12.2   
1              no               294.45                 7.7   
2              no                87.30                11.9   
3             yes               321.30                 9.5   
4              no               243.00                10.9   

   maximum_daily_mins  ...  maximum_days  customer_support_calls  churn  \
0               16.81  ...             4                       1      0   
1               33.37  ...            